# Combining Grounding DINO with Segment Anything (SAM) for text-based mask generation

In this notebook, we're going to combine 2 very cool models - [Grounding DINO](https://huggingface.co/docs/transformers/main/en/model_doc/grounding-dino) and [SAM](https://huggingface.co/docs/transformers/en/model_doc/sam). We'll use Grounding DINO to generate bounding boxes based on text prompts, after which we can prompt SAM to generate corresponding segmentation masks for them.

This is based on the popular [Grounded Segment Anything](https://github.com/IDEA-Research/Grounded-Segment-Anything) project - just with fewer lines of code as the models are now available in the Transformers library. Refer to the [paper](https://arxiv.org/abs/2401.14159) for details.

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/model_doc/grounded_sam.png"
alt="drawing" width="900"/>

<small> Grounded SAM overview. Taken from the <a href="https://github.com/IDEA-Research/Grounded-Segment-Anything">original repository</a>. </small>

Author of this notebook: [Eduardo Pacheco](https://huggingface.co/EduardoPacheco) - give him a follow on Hugging
 Face!

## Set-up environment

Let's start by installing 🤗 Transformers from source since Grounding DINO is brand new at the time of writing.

In [ ]:
# !pip install --upgrade -q git+https://github.com/huggingface/transformers
# !pip install --upgrade pip


StatementMeta(, 8e0b499d-634c-4227-b064-12912a5bedf1, 171, Finished, Available, Finished)

In [ ]:
# from notebookutils import mssparkutils
# mssparkutils.session.restartPython()

StatementMeta(, 8e0b499d-634c-4227-b064-12912a5bedf1, 172, Finished, Available, Finished)

## Imports

Let's start by importing the required libraries.

In [ ]:
import random
from dataclasses import dataclass
from typing import Any, List, Dict, Optional, Union, Tuple

# import cv2
import torch
import requests
import numpy as np
from PIL import Image
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from transformers import AutoModelForMaskedLM, AutoProcessor, AutoModelForZeroShotObjectDetection

StatementMeta(, 8e0b499d-634c-4227-b064-12912a5bedf1, 173, Finished, Available, Finished)

## Result Utils

We'll store the detection results of Grounding DINO in a dedicated Python dataclass.

In [ ]:
@dataclass
class BoundingBox:
    xmin: int
    ymin: int
    xmax: int
    ymax: int

    @property
    def xyxy(self) -> List[float]:
        return [self.xmin, self.ymin, self.xmax, self.ymax]

@dataclass
class DetectionResult:
    score: float
    label: str
    box: BoundingBox
    mask: Optional[np.array] = None

    @classmethod
    def from_dict(cls, detection_dict: Dict) -> 'DetectionResult':
        return cls(score=detection_dict['score'],
                   label=detection_dict['label'],
                   box=BoundingBox(xmin=detection_dict['box']['xmin'],
                                   ymin=detection_dict['box']['ymin'],
                                   xmax=detection_dict['box']['xmax'],
                                   ymax=detection_dict['box']['ymax']))

StatementMeta(, 8e0b499d-634c-4227-b064-12912a5bedf1, 174, Finished, Available, Finished)

## Plot Utils

Below, some utility functions are defined as we'll draw the detection results of Grounding DINO on top of the image.

In [ ]:
def annotate(image: Union[Image.Image, np.ndarray], detection_results: List[DetectionResult]) -> np.ndarray:
    # Convert to PIL Image if input is ndarray
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)
    
    # Create a drawing context
    draw = ImageDraw.Draw(image)

    # Iterate over detections and add bounding boxes and masks
    for detection in detection_results:
        label = detection.label
        score = detection.score
        box = detection.box
        mask = detection.mask

        # Sample a random color for each detection
        color = tuple(random.randint(0, 255) for _ in range(3))

        # Draw bounding box
        draw.rectangle([box.xmin, box.ymin, box.xmax, box.ymax], outline=color, width=2)
        draw.text((box.xmin, box.ymin - 10), f'{label}: {score:.2f}', fill=color)

        # If mask is available, apply it using matplotlib to display the contours
        if mask is not None:
            mask_image = Image.fromarray((mask * 255).astype(np.uint8))
            plt.imshow(image)
            plt.imshow(mask_image, alpha=0.3, cmap="jet")
            plt.axis('off')
            plt.show()
    
    # Convert the image back to numpy array if needed
    return np.array(image)
def plot_detections(
    image: Union[Image.Image, np.ndarray],
    detections: List[DetectionResult],
    save_name: Optional[str] = None
) -> None:
    annotated_image = annotate(image, detections)
    plt.imshow(annotated_image)
    plt.axis('off')
    if save_name:
        plt.savefig(save_name, bbox_inches='tight')
    plt.show()

StatementMeta(, 8e0b499d-634c-4227-b064-12912a5bedf1, 175, Finished, Available, Finished)

In [ ]:
def random_named_css_colors(num_colors: int) -> List[str]:
    """
    Returns a list of randomly selected named CSS colors.

    Args:
    - num_colors (int): Number of random colors to generate.

    Returns:
    - list: List of randomly selected named CSS colors.
    """
    # List of named CSS colors
    named_css_colors = [
        'aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanchedalmond',
        'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue',
        'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgreen', 'darkgrey',
        'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen',
        'darkslateblue', 'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue',
        'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro', 'ghostwhite',
        'gold', 'goldenrod', 'gray', 'green', 'greenyellow', 'grey', 'honeydew', 'hotpink', 'indianred', 'indigo', 'ivory',
        'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 'lightcoral', 'lightcyan', 'lightgoldenrodyellow',
        'lightgray', 'lightgreen', 'lightgrey', 'lightpink', 'lightsalmon', 'lightseagreen', 'lightskyblue', 'lightslategray',
        'lightslategrey', 'lightsteelblue', 'lightyellow', 'lime', 'limegreen', 'linen', 'magenta', 'maroon', 'mediumaquamarine',
        'mediumblue', 'mediumorchid', 'mediumpurple', 'mediumseagreen', 'mediumslateblue', 'mediumspringgreen', 'mediumturquoise',
        'mediumvioletred', 'midnightblue', 'mintcream', 'mistyrose', 'moccasin', 'navajowhite', 'navy', 'oldlace', 'olive',
        'olivedrab', 'orange', 'orangered', 'orchid', 'palegoldenrod', 'palegreen', 'paleturquoise', 'palevioletred', 'papayawhip',
        'peachpuff', 'peru', 'pink', 'plum', 'powderblue', 'purple', 'rebeccapurple', 'red', 'rosybrown', 'royalblue', 'saddlebrown',
        'salmon', 'sandybrown', 'seagreen', 'seashell', 'sienna', 'silver', 'skyblue', 'slateblue', 'slategray', 'slategrey',
        'snow', 'springgreen', 'steelblue', 'tan', 'teal', 'thistle', 'tomato', 'turquoise', 'violet', 'wheat', 'white',
        'whitesmoke', 'yellow', 'yellowgreen'
    ]

    # Sample random named CSS colors
    return random.sample(named_css_colors, min(num_colors, len(named_css_colors)))

def plot_detections_plotly(
    image: np.ndarray,
    detections: List[DetectionResult],
    class_colors: Optional[Dict[str, str]] = None
) -> None:
    # If class_colors is not provided, generate random colors for each class
    if class_colors is None:
        num_detections = len(detections)
        colors = random_named_css_colors(num_detections)
        class_colors = {}
        for i in range(num_detections):
            class_colors[i] = colors[i]


    fig = px.imshow(image)

    # Add bounding boxes
    shapes = []
    annotations = []
    for idx, detection in enumerate(detections):
        label = detection.label
        box = detection.box
        score = detection.score
        mask = detection.mask

        polygon = mask_to_polygon(mask)

        fig.add_trace(go.Scatter(
            x=[point[0] for point in polygon] + [polygon[0][0]],
            y=[point[1] for point in polygon] + [polygon[0][1]],
            mode='lines',
            line=dict(color=class_colors[idx], width=2),
            fill='toself',
            name=f"{label}: {score:.2f}"
        ))

        xmin, ymin, xmax, ymax = box.xyxy
        shape = [
            dict(
                type="rect",
                xref="x", yref="y",
                x0=xmin, y0=ymin,
                x1=xmax, y1=ymax,
                line=dict(color=class_colors[idx])
            )
        ]
        annotation = [
            dict(
                x=(xmin+xmax) // 2, y=(ymin+ymax) // 2,
                xref="x", yref="y",
                text=f"{label}: {score:.2f}",
            )
        ]

        shapes.append(shape)
        annotations.append(annotation)

    # Update layout
    button_shapes = [dict(label="None",method="relayout",args=["shapes", []])]
    button_shapes = button_shapes + [
        dict(label=f"Detection {idx+1}",method="relayout",args=["shapes", shape]) for idx, shape in enumerate(shapes)
    ]
    button_shapes = button_shapes + [dict(label="All", method="relayout", args=["shapes", sum(shapes, [])])]

    fig.update_layout(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        # margin=dict(l=0, r=0, t=0, b=0),
        showlegend=True,
        updatemenus=[
            dict(
                type="buttons",
                direction="up",
                buttons=button_shapes
            )
        ],
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Show plot
    fig.show()


StatementMeta(, 8e0b499d-634c-4227-b064-12912a5bedf1, 176, Finished, Available, Finished)

## Utils

In [ ]:
def mask_to_polygon(mask: np.ndarray) -> List[List[int]]:
    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    largest_contour = max(contours, key=cv2.contourArea)

    # Extract the vertices of the contour
    polygon = largest_contour.reshape(-1, 2).tolist()

    return polygon

def polygon_to_mask(polygon: List[Tuple[int, int]], image_shape: Tuple[int, int]) -> np.ndarray:
    """
    Convert a polygon to a segmentation mask.

    Args:
    - polygon (list): List of (x, y) coordinates representing the vertices of the polygon.
    - image_shape (tuple): Shape of the image (height, width) for the mask.

    Returns:
    - np.ndarray: Segmentation mask with the polygon filled.
    """
    # Create an empty mask
    mask = np.zeros(image_shape, dtype=np.uint8)

    # Convert polygon to an array of points
    pts = np.array(polygon, dtype=np.int32)

    # Fill the polygon with white color (255)
    cv2.fillPoly(mask, [pts], color=(255,))

    return mask
# load the image from the url or local path
def load_image(image_str: str) -> Image.Image:
    if image_str.startswith("http"):
        image = Image.open(requests.get(image_str, stream=True).raw).convert("RGB")
    else:
        image = Image.open(image_str).convert("RGB")

    return image
# get the boxes from the detection results
def get_boxes(results: DetectionResult) -> List[List[List[float]]]:
    boxes = []
    for result in results:
        xyxy = result.box.xyxy
        boxes.append(xyxy)

    return [boxes]

def refine_masks(masks: torch.BoolTensor, polygon_refinement: bool = False) -> List[np.ndarray]:
    masks = masks.cpu().float()
    masks = masks.permute(0, 2, 3, 1)
    masks = masks.mean(axis=-1)
    masks = (masks > 0).int()
    masks = masks.numpy().astype(np.uint8)
    masks = list(masks)

    if polygon_refinement:
        for idx, mask in enumerate(masks):
            shape = mask.shape
            polygon = mask_to_polygon(mask)
            mask = polygon_to_mask(polygon, shape)
            masks[idx] = mask

    return masks

StatementMeta(, 8e0b499d-634c-4227-b064-12912a5bedf1, 177, Finished, Available, Finished)

## Grounded Segment Anything (SAM)

Now it's time to define the Grounded SAM approach!

The approach is very simple:
1. use Grounding DINO to detect a given set of texts in the image. The output is a set of bounding boxes.
2. prompt Segment Anything (SAM) with the bounding boxes, for which the model will output segmentation masks.

In [ ]:
def detect(
    image: Image.Image,
    labels: List[str],
    threshold: float = 0.3,
    detector_id: Optional[str] = None
) -> List[Dict[str, Any]]:
    """
    Use Grounding DINO to detect a set of labels in an image in a zero-shot fashion.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    detector_id = detector_id if detector_id is not None else "IDEA-Research/grounding-dino-base"
    processor = AutoProcessor.from_pretrained(detector_id)
    object_detector = AutoModelForZeroShotObjectDetection.from_pretrained(detector_id).to(device)

    # add a dot to the end of each label
    labels = [label if label.endswith(".") else label+"." for label in labels]
    text = " ".join(labels)
    # text = "a cat. a remote control."
    # image_url = "http://images.cocodataset.org/val2017/000000039769.jpg"
    # image = Image.open(requests.get(image_url, stream=True).raw)
    
    # process the image and text
    inputs = processor(images=image, text=text, return_tensors="pt").to(device)
    with torch.no_grad():
        results = object_detector(**inputs)
    # post process the results
    results = processor.post_process_grounded_object_detection(
        results,
        inputs.input_ids,
        box_threshold=threshold,
        text_threshold=threshold,
        target_sizes=[image.size[::-1]]
    )
    # print(results)
    return results

def segment(
    image: Image.Image,
    detection_results: List[Dict[str, Any]],
    polygon_refinement: bool = False,
    segmenter_id: Optional[str] = None
) -> List[DetectionResult]:
    """
    Use Segment Anything (SAM) to generate masks given an image + a set of bounding boxes.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    segmenter_id = segmenter_id if segmenter_id is not None else "facebook/sam-vit-base"
    boxes = get_boxes(detection_results)
    # print(boxes)
    segmentator = AutoModelForMaskedLM.from_pretrained(segmenter_id).to(device)
    processor = AutoProcessor.from_pretrained(segmenter_id)
    inputs = processor(images=image, input_boxes=boxes, return_tensors="pt").to(device)
    # forward the model
    outputs = segmentator(**inputs)
    # post process the masks
    masks = processor.post_process_masks(
        masks=outputs.pred_masks,
        original_sizes=inputs.original_sizes,
        reshaped_input_sizes=inputs.reshaped_input_sizes
    )[0]
    # refine the masks
    masks = refine_masks(masks, polygon_refinement)
    # update the detection results
    for detection_result, mask in zip(detection_results, masks):
        detection_result.mask = mask

    return detection_results

def grounded_segmentation(
    image: Union[Image.Image, str],
    labels: List[str],
    threshold: float = 0.3,
    polygon_refinement: bool = False,
    detector_id: Optional[str] = None,
    segmenter_id: Optional[str] = None
) -> Tuple[np.ndarray, List[DetectionResult]]:
    
    image = load_image(image)

    detections = detect(image, labels, threshold, detector_id)
    # detections = segment(image, detections, polygon_refinement, segmenter_id)

    return np.array(image), detections

StatementMeta(, 8e0b499d-634c-4227-b064-12912a5bedf1, 178, Finished, Available, Finished)

### Inference

Let's showcase Grounded SAM on our favorite image: the cats image from the COCO dataset.

In [ ]:
# get image data
# taskID = 14
df = spark.sql(f"SELECT * FROM InFreGen.pic WHERE state = '1' and taskID = {taskID} ")
data = df.select("taskID", "picID", "curPicPath", "keywords").toPandas()
data

StatementMeta(, 8e0b499d-634c-4227-b064-12912a5bedf1, 182, Finished, Available, Finished)

,taskID,picID,curPicPath,keywords
0,14,4,/lakehouse/default/Files/images/14/4_small.jpg,sheep
1,14,3,/lakehouse/default/Files/images/14/3_small.jpg,sheep
2,14,2,/lakehouse/default/Files/images/14/2_small.jpg,sheep
3,14,1,/lakehouse/default/Files/images/14/1_small.jpg,sheep
4,14,0,/lakehouse/default/Files/images/14/0_small.jpg,sheep


Let's visualize the results:

# Cutting

In [ ]:
import json
from PIL import Image

boundRes = {}
# threshold for the detection
threshold = 0.3
# detector id
detector_id = "IDEA-Research/grounding-dino-base"
segmenter_id = "facebook/sam-vit-base"

for index, image in data.iterrows():
    
    picID = image['picID']
    image_path = image['curPicPath']
    if(image_path == ""):
        continue
    labels = [image['keywords']]

    print(image_path)
    print(labels)
    try:
        # Perform grounded segmentation
        image_array, detections = grounded_segmentation(
            image=image_path,
            labels=labels,
            threshold=threshold,
            polygon_refinement=True,
            detector_id=detector_id,
            segmenter_id=segmenter_id
        )
    except:
        continue

    # Get the detection boxes and labels
    boxes = detections[0]["boxes"].tolist()
    labels_list = detections[0]["labels"]

    # Find the largest bounding box based on area
    max_box = None
    max_area = 0

    print(detections[0]["scores"])
    for box in boxes:
        x_min, y_min, x_max, y_max = box
        area = (x_max - x_min) * (y_max - y_min)
        if area > max_area:
            max_area = area
            max_box = box

    # If a bounding box was found, crop the image
    if max_box:
        x_min, y_min, x_max, y_max = max_box
        with Image.open(image_path) as img:
            cropped_img = img.crop((x_min, y_min, x_max, y_max))

            # Save the cropped image with '_cutted' suffix
            output_path = image_path.replace(".jpg", "_cutted.jpg")
            print(output_path + " saved")
            cropped_img.save(output_path)

        # Update boundRes with the bounding box info and output path
        boundRes[picID] = {
            "boxes": boxes,
            "labels": labels_list,
            "largest_box": max_box,
            "cropped_image_path": output_path
        }

        spark.sql(f"UPDATE InFreGen.pic SET state = '3', curPicPath = '{output_path}' WHERE taskID = '{taskID}' AND picID = '{picID}'")


    # break
    # Optionally, convert the result to JSON format for saving or output
    # json_output = json.dumps(boundRes, indent=4)
    # print(json_output)


StatementMeta(, 8e0b499d-634c-4227-b064-12912a5bedf1, 190, Finished, Available, Finished)

/lakehouse/default/Files/images/14/4_small.jpg
['sheep']
tensor([0.9506])
/lakehouse/default/Files/images/14/4_small_cutted.jpg saved
/lakehouse/default/Files/images/14/3_small.jpg
['sheep']
tensor([0.7347, 0.7171])
/lakehouse/default/Files/images/14/3_small_cutted.jpg saved
/lakehouse/default/Files/images/14/2_small.jpg
['sheep']
/lakehouse/default/Files/images/14/1_small.jpg
['sheep']
tensor([0.8800])
/lakehouse/default/Files/images/14/1_small_cutted.jpg saved
/lakehouse/default/Files/images/14/0_small.jpg
['sheep']
